### Data Loading
the data will be called from 'src/loaders.py' file with the required functions. We'll use these functions to load and visualize the data.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from typing import List, Tuple, Dict

# Assuming these functions are implemented in src/loaders.py
from src.loaders import load_mat, load_point_clouds, get_mags
from src.visualize import visualize

# Load clean and noisy data
clean_voxel_data, _ = load_mat('clean_data.mat')
noisy_voxel_data, _ = load_mat('noisy_data.mat')
point_clouds, _ = load_point_clouds('clean_data')

# Convert complex values to magnitudes
clean_voxel_data = get_mags(clean_voxel_data)
noisy_voxel_data = get_mags(noisy_voxel_data)

# Visualize data (optional)
visualize(clean_voxel_data[0], title='Clean Data')
visualize(noisy_voxel_data[0], title='Noisy Data')


### Preprocessing Data
Defining a custom dataset class to handle the voxel data and corresponding lines.

In [3]:
class VoxelDataset(Dataset):
    def __init__(self, voxel_data, lines):
        self.voxel_data = voxel_data
        self.lines = lines

    def __len__(self):
        return len(self.voxel_data)

    def __getitem__(self, idx):
        return self.voxel_data[idx], self.lines[idx]



## We will apply Hough Transform to get ground truth lines for clean data
### Assuming apply_hough_transform is defined elsewhere

In [ ]:
def apply_hough_transform(voxel_data):
    '''we can define the code here for hough transformation'''
    lines = np.random.rand(len(voxel_data), 6)  # Example format: [npoints, a1, a2, a3, b1, b2, b3]
    return lines

clean_lines = apply_hough_transform(clean_voxel_data)

# Create datasets and dataloaders
clean_dataset = VoxelDataset(clean_voxel_data, clean_lines)
clean_data_loader = DataLoader(clean_dataset, batch_size=32, shuffle=True)

noisy_dataset = VoxelDataset(noisy_voxel_data, clean_lines)  # using clean_lines as target
noisy_data_loader = DataLoader(noisy_dataset, batch_size=32, shuffle=True)


### Defining a Model
### I chose a 3D CNN model to process the voxel data.

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class HoughNN(nn.Module):
    def __init__(self):
        super(HoughNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 6)  # Output size: [npoints, a1, a2, a3, b1, b2, b3]

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Training the Model
### Defining the training function and training the model using clean data first.

In [7]:
import torch.optim as optim

def train_model(model, dataloader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in dataloader:
            inputs = inputs.unsqueeze(1).float()  # we have to add channel dimension
            targets = targets.float()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch}/{num_epochs}, Loss: {epoch_loss:.4f}')

    return model

### Initializing the model

In [ ]:
model = HoughNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


### Training the model on clean data

In [ ]:
print("Training on clean data...")
model = train_model(model, clean_data_loader, criterion, optimizer, num_epochs=25)

### Evaluating Model
### Evaluating the model on noisy data and comparing it with the Hough Transform.

In [ ]:
def evaluate_model(model, dataloader, criterion):
    model.eval()
    mse = 0.0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.unsqueeze(1).float()
            targets = targets.float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            mse += loss.item() * inputs.size(0)

    mse /= len(dataloader.dataset)
    return mse

# Evaluate the model on noisy data
print("Evaluating on noisy data...")
noisy_mse = evaluate_model(model, noisy_data_loader, criterion)
print(f"Mean Squared Error on noisy data: {noisy_mse:.4f}")

# Evaluate Hough Transform on noisy data
noisy_lines = apply_hough_transform(noisy_voxel_data)
hough_mse = ((noisy_lines - clean_lines)**2).mean().item()

print(f"Hough Transform MSE on noisy data: {hough_mse:.4f}")
